In [7]:
import sys;
import os;
import glob;
import math;
import random;
import torch;
import torch.nn as nn;
import numpy as np
import copy
sys.path.append(os.path.abspath("../"))

In [2]:
from tinynn.converter import TFLiteConverter

In [8]:
from datetime import datetime

In [9]:
from th.resources.models import *

In [4]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [5]:
pt_uncompressed_model = "./trained_models/current_best/acdnet_alarm_3rd_20240119093633_acc_97.7272720336914_190th_epoch.pt"
tflite_save_path = "./tflite_models/acdnet_alarm_3rd_acc97.7_{}.tflite"

In [10]:
inputLength = 30225
nClass = 2;
sr = 20000

In [16]:
"""
alarm detect(two class) model config
[8, 64, 32, 64, 64, 128, 128, 256, 256, 512, 512, 2]
"""

'\nalarm detect(two class) model config\n[8, 64, 32, 64, 64, 128, 128, 256, 256, 512, 512, 2]\n'

In [12]:
state = torch.load(pt_uncompressed_model, map_location='cpu');
print(state['config']);
net = GetACDNetModel(input_len=inputLength, nclass=nClass, sr=sr, channel_config=state['config']).to('cpu');

[8, 64, 32, 64, 64, 128, 128, 256, 256, 512, 512, 2]


In [15]:
with torch.no_grad():
            # dummy_input = torch.randn(1, 1, 30225, 1); wrong: RuntimeError: quantized::conv2d (qnnpack): each dimension of output tensor should be greater than 0.
            dummy_input = torch.randn(1, 1, 1, 30225); #correct,workable
            # dummy_input = torch.randn(30225,1,1,1); wrong: RuntimeError: quantized::conv2d (qnnpack): each dimension of output tensor should be greater than 0.
            # dummy_input = torch.randn(1,30225,1,1); wrong:RuntimeError: Input channel size of weight and bias must match.
            converter = TFLiteConverter(net,
                                        dummy_input,
                                        tflite_path=tflite_save_path.format(genDataTimeStr()))
            converter.convert()

INFO (tinynn.converter.base) Generated model saved to ./tflite_models/acdnet_alarm_3rd_acc97.7_20240119121437.tflite
